## Extract Transform and Load the Data set.

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

reviews_df = pd.read_csv("Reviews.csv")

helpfulnessMask = reviews_df["HelpfulnessNumerator"] > 0
reviews_df = reviews_df[helpfulnessMask]

reviews_df['ReviewLength'] = list(map(len, reviews_df['Text']))
reviewLengthMask = reviews_df["ReviewLength"] <= 512
reviews_df = reviews_df[reviewLengthMask]

reviews_df.drop(columns=["ProductId", "ProfileName", "HelpfulnessDenominator", "Time", "Summary"], inplace=True)

print(reviews_df.shape)
reviews_df.head()

(180933, 6)


,Id,UserId,HelpfulnessNumerator,Score,Text,ReviewLength
0,1,A3SGXH7AUHU8GW,1,5,I have bought several of the Vitality canned d...,263
2,3,ABXLMWJIXXAIN,1,4,This is a confection that has been around a fe...,509
3,4,A395BORC6FGVXV,3,2,If you are looking for the secret ingredient i...,219
8,9,A1MZYO9TZK0BBI,1,5,Right now I'm mostly just sprouting this so my...,131
11,12,A2725IB4YY9JEB,4,5,One of my boys needed to lose some weight and ...,328


## Sentiment Analysis

In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import pipeline
from scipy.special import softmax

# sentiment_analysis_pipeline = pipeline('sentiment-analysis')

# x = sentiment_analysis_pipeline('Hello I am happy')
# print(x[0]['label'])
# reviews_df['Label'] = sentiment_analysis_pipeline(reviews_df['Text'])
# reviews_df['Label'] = list(map(sentiment_analysis_pipeline, reviews_df['Text']))

# x = list(map(sentiment_analysis_pipeline, reviews_df['Text']))

# print(x)

# reviews_df



#reviews_df.to_csv("transformed_reviews.csv", header=True, index=True)

**Random Sample of 50 Positive and 50 Negative Reviews for T-test**

In [17]:
pos_reviews = reviews_df["Score"] >= 4 # 4 stars or more is a positive review
reviews_pos = reviews_df[pos_reviews]
neg_reviews = reviews_df["Score"] <= 2 # 2 stars and below is a negative review
reviews_neg = reviews_df[neg_reviews]

reviews_pos.drop(columns=["Text", "ReviewLength"], inplace=True)
reviews_neg.drop(columns=["Text", "ReviewLength"], inplace=True)

print(reviews_pos.shape)
reviews_pos.sample(50)

(138556, 4)


C:\Users\Utsav\AppData\Local\Temp\ipykernel_7048\7178374.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_pos.drop(columns=["Text", "ReviewLength"], inplace=True)
C:\Users\Utsav\AppData\Local\Temp\ipykernel_7048\7178374.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_neg.drop(columns=["Text", "ReviewLength"], inplace=True)


,Id,UserId,HelpfulnessNumerator,Score
188127,188128,A2TOA9C1W4MMQ4,1,5
78448,78449,A1AU6XU1K1QVHN,1,5
115005,115006,A1ATKZ120NAGUF,1,4
53184,53185,A2BU8RXVQ32R8P,3,5
385007,385008,A2BFGQLYQXJV9A,2,5
53316,53317,A16MJNKAJ7ISXT,1,5
102720,102721,A32PKGUGQ3JUYQ,9,5
192050,192051,A2Y8TZI9LNIEJ6,1,5
155743,155744,A16V6Y0BWZAGNI,3,5
557379,557380,A1TNKFJBUL0ZAV,2,5


In [18]:
print(reviews_neg.shape)
reviews_neg.sample(50)

(30832, 4)


,Id,UserId,HelpfulnessNumerator,Score
531531,531532,A2Z9VE09ZBS4YE,2,1
252003,252004,A37JMHPU2JBHLC,1,1
509295,509296,A2KEES9I8NTSTP,2,1
3662,3663,AZ62H2NKUXB2W,1,1
46659,46660,A3VE9PDBSDVFS1,1,1
542709,542710,A14XYYTO6H9ZYI,1,1
410749,410750,A11D5K3TRMW1ZA,2,2
273382,273383,A11ETZNY80TQ2E,2,1
49409,49410,A1R7F3H4D9KTIW,2,1
469703,469704,ALJ9JSCH5YEU,1,1
